# Setup

You might need to run the cell below to setup data required by the Luminesce commands.


In [ ]:
# Import general modules
import logging
import lusid
import lusid.models as models
import json
import argparse

# Import LUSID Drive modules
from lusid.utilities import ApiClientFactory as LusidApiClientFactory
from lusidjam import RefreshingToken

# Create loggers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


def create_recipe(api_factory):
    recipes_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

    scope = "luminesce-examples"
    recipe_code = "marketValue"

    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="100D.0D",
                ),
                models.MarketDataKeyRule(
                    key="FX.*.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="100D.0D",
                ),
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Client",
                credit="Client",
                equity="Client",
                fx="Client",
                rates="Client",
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
                attempt_to_infer_missing_fx=True,
            ),
        ),
        pricing=models.PricingContext(
            model_rules=[
                models.VendorModelRule(
                    supplier="Lusid", model_name="SimpleStatic", instrument_type="Bond"
                )
            ]
        ),
    )

    upsert_configuration_recipe_response = recipes_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )

    print(upsert_configuration_recipe_response)


def create_txn_types(api_factory):
    system_configuration = api_factory.build(lusid.api.SystemConfigurationApi)

    response = system_configuration.set_transaction_configuration_source(
        source="abor",
        set_transaction_configuration_source_request=[
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsIn",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Subscription",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=1,
                        movement_options=["Capital"],
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsOut",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Shorter",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Redemption",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=-1,
                        movement_options=["Capital"],
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Buy",
                        description="Buy",
                        transaction_class="Buy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Bought",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashInvested",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Sell",
                        description="Sell",
                        transaction_class="Sell",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Sale",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashProceeds",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=1,
                        movement_options=[],
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FxSpotBuy",
                        description="FxSpotBuy",
                        transaction_class="FxSpotBuy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotBuyLeg",
                        movement_types="CashCommitment",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotSellLeg",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                    ),
                ],
            ),
        ],
    )

    print(response)


if __name__ == "__main__":
    token = RefreshingToken()

    if token is not None:
        lusid_api_factory = LusidApiClientFactory(token=token)

    else:
        ap = argparse.ArgumentParser()
        ap.add_argument("-s", "--secrets", type=str, help="full path to json file")
        args = ap.parse_args()
        secrets_file = args.secrets
        lusid_api_factory = LusidApiClientFactory(api_secrets_filename=secrets_file)

    create_recipe(lusid_api_factory)
    create_txn_types(lusid_api_factory)

# Abor

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 01: Create a portfolio

In [ ]:
%%luminesce

/*

------------------------------
Create a Transaction portfolio
------------------------------

In this snippet we load a Transaction portfolio into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the portfolio details

@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';
@@writeAction = select 'Upsert';

@createPortfolio = select 'Transaction' as PortfolioType,
@@scope as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@scope  as InstrumentScopes,
@@portfolioCode as DisplayName,
@@portfolioCode  as Description,
#2000-01-01# as Created,
'' as SubHoldingKeys,
'GBP' as BaseCurrency,
@@writeAction as WriteAction
;

-- Step 2: Load portfolio into LUSID

select *
from Lusid.Portfolio.Writer
where ToWrite = @createPortfolio;


#### Step 02: Create equity instruments

In [ ]:
%%luminesce

/*

------------------------------
Create some equity instruments
------------------------------

In this snippet we create some Equity instruments into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the equity instruments

@@scope = select 'luminesce-examples';

@instrumentsData= 
values
('Tesco', 'FBNABOR001', 'GBP'),
('Sainsburys', 'FBNABOR002', 'GBP'),
('Walmart', 'FBNABOR003', 'USD'),
('Wholefoods', 'FBNABOR004', 'USD'),
('Waitrose', 'FBNABOR005', 'GBP');

@instrumentsForUpload = select
column1 as DisplayName,
column2 as ClientInternal,
column3 as DomCcy,
@@scope as Scope
from @instrumentsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @instrumentsForUpload;


#### Step 03: Create bond instruments

In [ ]:
%%luminesce

/*

-----------------------
Create Bond instruments
-----------------------

In this snippet we create Bond instruments.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the bond instruments

@bondsData= 
values
('US BOND 4% 01/01/2033', 'FBNBND001',  #2023-01-01#,  0.04, 'USD', '1Y', 'ActAct', #2033-01-01#),
('US BOND 7% 01/01/2028', 'FBNBND002',  #2023-01-01#,  0.07, 'USD', '1Y', 'ActAct', #2028-01-01#),
('UK BOND 3% 01/01/2033', 'FBNBND003',  #2023-01-01#,  0.03, 'GBP', '1Y', 'ActAct', #2033-01-01#),
('UK BOND 8% 01/01/2028', 'FBNBND004',  #2023-01-01#,  0.08, 'GBP', '1Y', 'ActAct', #2028-01-01#);


@bondsDataForUpload = 
select
Column1 as DisplayName,
column2 as ClientInternal,
column3 as StartDate,
column4 as CouponRate,
column5 as DomCcy,
column5 as FlowConventionsCurrency,
column6 as FlowConventionsPaymentFrequency,
column7 as FlowConventionsDayCountConvention,
'MF' as FlowConventionsRollConvention,
column5 as FlowConventionsPaymentCalendars,
column5 as FlowConventionsResetCalendars,
0 as FlowConventionsSettleDays,
0 as FlowConventionsResetDays,
1 as Principal,
column8 as MaturityDate,
@@scope as Scope
from @bondsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Bond.Writer
where ToWrite = @bondsDataForUpload;


#### Step 04: Create instrument properties

In [ ]:
%%luminesce

/*

----------------------------
Create instrument properties
----------------------------

In this snippet we create some instrument properties.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the property definitions

@newProperties =
values
('Instrument', @@scope, 'Sector', 'string'),
('Instrument', @@scope, 'Country', 'string'),
('Instrument', @@scope, 'InternalRating', 'number');

@propertyDefinitions =
select 
Column1 as [Domain], 
Column2 as [PropertyScope], 
Column3 as [PropertyCode], 
Column3 as [DisplayName], 
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
column4 as [DataTypeCode]
from @newProperties;

-- Step 2: Load property definitions

select *
from Lusid.Property.Definition.Writer
where ToWrite = @propertyDefinitions;


#### Step 05: Assign properties to instrument

In [ ]:
%%luminesce

/*

--------------------------------
Assign properties to instruments
--------------------------------

In this snippet we assign properties to instruments.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup the property definitions referenced below 
    2. Setup the instruments referenced below

*/

@@scope = select 'luminesce-examples';

-- Step 1: Define the property definitions

@newProperties =
values

-- Sectors
('FBNABOR001', @@scope, 'Sector', 'Consumer'),
('FBNABOR002', @@scope, 'Sector', 'Consumer'),
('FBNABOR003', @@scope, 'Sector', 'Consumer'),
('FBNABOR004', @@scope, 'Sector', 'Consumer'),
('FBNABOR005', @@scope, 'Sector', 'Consumer'),
('FBNBND001', @@scope, 'Sector', 'Consumer'),
('FBNBND002', @@scope, 'Sector', 'Consumer'),
('FBNBND003', @@scope, 'Sector', 'Consumer'),
('FBNBND004', @@scope, 'Sector', 'Consumer'),

-- Country
('FBNABOR001', @@scope, 'Country', 'United Kingdom'),
('FBNABOR002', @@scope, 'Country', 'United Kingdom'),
('FBNABOR003', @@scope, 'Country', 'United States'),
('FBNABOR004', @@scope, 'Country', 'United States'),
('FBNABOR005', @@scope, 'Country', 'United Kingdom'),
('FBNBND001', @@scope, 'Country', 'United States'),
('FBNBND002', @@scope, 'Country', 'United States'),
('FBNBND003', @@scope, 'Country', 'United Kingdom'),
('FBNBND004', @@scope, 'Country', 'United Kingdom'),

-- Internal ratings
('FBNABOR001', @@scope, 'InternalRating', 8),
('FBNABOR002', @@scope, 'InternalRating', 8),
('FBNABOR003', @@scope, 'InternalRating', 9),
('FBNABOR004', @@scope, 'InternalRating', 7),
('FBNABOR005', @@scope, 'InternalRating', 7),
('FBNBND001', @@scope, 'InternalRating', 8),
('FBNBND002', @@scope, 'InternalRating', 8),
('FBNBND003', @@scope, 'InternalRating', 6),
('FBNBND004', @@scope, 'InternalRating', 7);

@instProperties =
select 
column1 as EntityId, 
'ClientInternal' as EntityIdType, 
'Instrument' as Domain,
Column2 as PropertyScope, 
Column3 as PropertyCode,
Column4 as Value,
@@scope as EntityScope
from @newProperties;

-- Upload the transformed data into LUSID

select *
from Lusid.Property.Writer
where ToWrite = @instProperties
limit 5;


#### Step 06: Create transactions

In [ ]:
%%luminesce

/*

------------------
Create transactions
------------------

In this snippet we create some Transactions.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a portfolio with scope/code per below
    2. Setup Buy and Sell Transaction Types
    3. Setup instruments with the ClientInternal instrument IDs referenced below

*/


@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';

-- Step 1: Define some transactions

@transactions = 
values

-- Equity Transactions
(@@scope, @@portfolioCode, 'txn_001', 'Buy', '2023-01-01', '2023-01-02', 1000, 10, 10000, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_002', 'Buy', '2023-01-01', '2023-01-02', 2000, 12, 10000, 'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_003', 'Buy', '2023-01-01', '2023-01-02', 3000, 13, 10000, 'USD', 'FBNABOR003', 0.8),
(@@scope, @@portfolioCode, 'txn_004', 'Buy', '2023-01-01', '2023-01-02', 4000, 14, 10000, 'USD', 'FBNABOR004', 0.8),
(@@scope, @@portfolioCode, 'txn_005', 'Buy', '2023-01-01', '2023-01-02', 5000, 15, 10000, 'GBP', 'FBNABOR005', 1),
(@@scope, @@portfolioCode, 'txn_006', 'Sell', '2023-02-01', '2023-02-02', 1000, 21, 21000, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_007', 'Sell', '2023-02-01', '2023-02-02', 1000, 22, 22000, 'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_008', 'Sell', '2023-02-01', '2023-02-02', 1000, 23, 23000, 'USD', 'FBNABOR003', 0.78),
(@@scope, @@portfolioCode, 'txn_009', 'Sell', '2023-02-01', '2023-02-02', 1000, 24, 24000, 'USD', 'FBNABOR004', 0.78),
(@@scope, @@portfolioCode, 'txn_010', 'Sell', '2023-02-01', '2023-02-02', 1000, 25, 25000, 'GBP', 'FBNABOR005', 1),

--Bond transactions
(@@scope, @@portfolioCode, 'txn_011', 'Buy', '2023-01-01', '2023-01-02', 100000, 100, 100000, 'USD', 'FBNBND001', 0.8),
(@@scope, @@portfolioCode, 'txn_012', 'Buy', '2023-01-01', '2023-01-02', 200000, 98, 200000, 'GBP', 'FBNBND003', 1),
(@@scope, @@portfolioCode, 'txn_013', 'Sell', '2023-02-01', '2023-02-02', 50000, 100, 50000, 'USD', 'FBNBND001', 0.78),
(@@scope, @@portfolioCode, 'txn_014', 'Sell', '2023-02-01', '2023-02-02', 100000, 102, 100000, 'GBP', 'FBNBND003', 1);


-- Step 2: Load transactions into LUSID

@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as ClientInternal,
column12 as TradeToPortfolioRate
from @transactions;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;


#### Step 07: Upload instrument prices

In [ ]:
%%luminesce

/*

------------------------
Upload instrument prices
------------------------

In this snippet we upload instrument prices into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup instruments with the ClientInternal instrument IDs referenced below


*/

@@scope = select 'luminesce-examples';
@@quoteDate = select #2023-03-03#;

-- Step 1: Define some instrument prices

@instrumentPriceData= 
values
('FBNABOR001', @@quoteDate, 31, 'GBP', 1),
('FBNABOR002', @@quoteDate, 32, 'GBP', 1),
('FBNABOR003', @@quoteDate, 33, 'GBP', 1),
('FBNABOR004', @@quoteDate, 34, 'USD', 1),
('FBNABOR005', @@quoteDate, 34, 'GBP', 1),
('FBNBND001', @@quoteDate, 105, 'USD', 100),
('FBNBND002', @@quoteDate, 105, 'USD', 100),
('FBNBND003', @@quoteDate, 107, 'GBP', 100),
('FBNBND004', @@quoteDate, 98, 'GBP', 100);

-- Step 2: Load prices into LUSID

@pricesForUpload = select
'ClientInternal' as InstrumentIdType,
column1 as Instrumentid,
@@scope as QuoteScope,
'Price' as QuoteType,
'Lusid' as Provider,
'mid' as Field,
column2 as QuoteEffectiveAt,
column3 as Value,
column4 as Unit,
column5 as ScaleFactor
from @instrumentPriceData;

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @pricesForUpload;



#### Step 08: Upload fx rates

In [ ]:
%%luminesce

/*

---------------
Upload FX rates
---------------

In this snippet we upload FX rates into LUSID.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@scope = select 'luminesce-examples';
@@quoteDate = select #2023-03-03#;

-- Step 1: Define FX rates

@fxRatesData= 
values
('GBP/USD', @@quoteDate, 1.2344),
('GBP/EUR', @@quoteDate, 1.1444);

-- Step 2: Load FX rates into LUSID

@quotesForUpload = select
'CurrencyPair' as InstrumentIdType,
column1 as Instrumentid,
@@scope as QuoteScope,
'Rate' as QuoteType,
'Lusid' as Provider,
'mid' as Field,
column2 as QuoteEffectiveAt,
column3 as Value,
substr(column1, 5, 7) as Unit
from @fxRatesData;

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotesForUpload;



#### Step 09: Create cash transactions

In [ ]:
%%luminesce

/*

------------------
Create transaction
------------------

In this snippet we create some cash Transactions.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a portfolio with scope/code per below
    2. Setup FundsIn Transaction Types

*/


@@scope = select 'luminesce-examples';
@@portfolioCode = select 'aborPortfolio';



/*  
    Step 1: Define some cash transactions
    
    In the step below, we create a transaction type called AborFundsIn which is configured to create
    Capital movements in the Journal Entry

*/

@transactions = 
values
(@@scope, @@portfolioCode, 'csh_001', 'AborFundsIn', '2023-01-01', '2023-01-01', 1000000, 1, 1000000, 'GBP', 'CCY_GBP', 1),
(@@scope, @@portfolioCode, 'csh_002', 'AborFundsIn', '2023-01-01', '2023-01-01', 1000000, 1, 1000000, 'USD', 'CCY_USD', 0.79);


@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as LusidInstrumentId,
Column12 as TradeToPortfolioRate
from @transactions;

-- Step 2: Load transactions into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;


#### Step 10: Create a chart of accounts

In [ ]:
%%luminesce

/*

--------------------------
Create a Chart of Accounts
--------------------------

In this snippet we create a Chart of Accounts.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Step 1: Define the Chart of Accounts

@@scope = select 'luminesce-examples';
@@code = select 'standardChartOfAccounts';
@@name = select 'Standard Chart Of Accounts';
@@writeAction = select 'Upsert';

@chartOfAccounts =
select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
@@name as Name,
@@name as Description,
@@writeAction as WriteAction;

-- Step 2: Upload Chart of Account into LUSID

select * from Lusid.ChartOfAccounts.Writer where ToWrite = @chartOfAccounts;


#### Step 11: Add accounts to chart of accounts

In [ ]:
%%luminesce

/*

---------------------------------
Add Accounts to Chart of Accounts
---------------------------------

In this snippet we add Accounts to a Chart of Accounts.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Chart of Accounts with the scope/code referenced below

*/

-- Step 1: Define some accounts

@@scope = select 'luminesce-examples';
@@code = select 'standardChartOfAccounts';

@accounts = values
-- Investments
('A0001-Investments-UK', 'Investments', 'Asset', 'Manual', 'Active'),
('A0002-Investments-USA', 'Investments', 'Asset', 'Manual', 'Active'),
('A0003-Investments-General', 'Investments', 'Asset', 'Manual', 'Active'),

-- Cash, commitments and capital
('A0004-Cash', 'Cash', 'Asset', 'Manual', 'Active'),
('A0005-Commitments', 'Commitments', 'Asset', 'Manual', 'Active'),
('A0006-Capital', 'Capital', 'Capital', 'Manual', 'Active'),

-- Gains and Losses
('A0007-RealisedGains', 'RealisedGains', 'Revenue', 'Manual', 'Active'),
('A0008-UnrealisedGains', 'UnrealisedGains', 'Income', 'Manual', 'Active'),
('A0009-Accruals', 'UnrealisedGains', 'Income', 'Manual', 'Active'),

-- Unknown catch alls
('A0010-Unknown-NA', 'Investments', 'Asset', 'Manual', 'Active'),
('A0011-Unknown-PL', 'P&L Other', 'Revenue', 'Manual', 'Active'),
('A0011-Unknown-CA', 'Capital', 'Capital', 'Manual', 'Active');


@chartsOfAccountsAccounts = select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
column1 as AccountCode,
column2 as Description,
column3 as Type,
column4 as Control,
column5 as Status
from @accounts;

-- Step 2: Assign Accounts onto a ChartOfAccount

select * from Lusid.ChartOfAccounts.Account.Writer 
where ToWrite = @chartsOfAccountsAccounts;


#### Step 12: Create posting module

In [ ]:
%%luminesce

/*

----------------------
Create posting modules
----------------------

In this snippet we create a posting module.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Chart of Accounts with the scope/code referenced below

*/

-- Step 1: Define a posting module

@@scope = select 'luminesce-examples';
@@code = select 'standardPostingModule';
@@writeAction = select 'Upsert';

@postingModule = 
select @@scope as PostingModuleScope, 
@@code as PostingModuleCode,
@@scope as ChartOfAccountsScope, 
'Active' as Status,
'standardChartOfAccounts' as ChartOfAccountsCode,
'Daily NAV' as Name,
'Posting module for daily NAV' as Description,
@@writeAction as WriteAction;

-- Step 2: Upload posting module into LUSID

select * from Lusid.PostingModule.Writer where ToWrite = @postingModule;


#### Step 13: Create posting rules

In [ ]:
%%luminesce

/*

--------------------
Create posting rules
--------------------

In this snippet we create some posting rules.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup a Posting Module wuth the scope/code referenced below


*/

-- Step 1: Define posting rules

@@scope = select 'luminesce-examples';
@@code = select 'standardPostingModule';

@rules_filters = values
(
    'Rule-001', 
    'EconomicBucket startswith ''NA'' 
        and HoldType eq ''P'' 
        and Properties[Instrument/luminesce-examples/Country] eq ''United Kingdom''',
    'A0001-Investments-UK'
),
(
    'Rule-002', 
    'EconomicBucket startswith ''NA'' 
        and HoldType eq ''P'' 
        and Properties[Instrument/luminesce-examples/Country] eq ''United States''',
    'A0002-Investments-USA'
),
(
    'Rule-003', 
    'EconomicBucket startswith ''NA'' 
        and HoldType eq ''P'' 
        and Properties[Instrument/luminesce-examples/Country] exists',
    'A0003-Investments-General'
),
(
    'Rule-004',
    'EconomicBucket startswith ''NA'' and HoldType eq ''B''',
    'A0004-Cash'
),
(
    'Rule-005',
    'EconomicBucket startswith ''NA'' and HoldType eq ''C''',
    'A0005-Commitments'),
(
    'Rule-006',
    'EconomicBucket eq ''CA_Capital''', 
    'A0006-Capital'
),
(
    'Rule-007', 
    'EconomicBucket startswith ''PL_Real''', 
    'A0007-RealisedGains'
),
(
    'Rule-008', 
    'EconomicBucket startswith ''PL_Unreal''',
    'A0008-UnrealisedGains'
),
(
    'Rule-009', 
    'EconomicBucket startswith ''PL_Accrued''',
    'A0009-Accruals'
),
(
    'Rule-010', 
    'EconomicBucket startswith ''NA''',
    'A0010-Unknown-NA'
),
(
    'Rule-011', 
    'EconomicBucket startswith ''CA''',
    'A0011-Unknown-CA'
)
;

-- Step 2: Add posting rules to posting module

@postingRules = select 
@@scope as PostingModuleScope,
@@code as PostingModuleCode,
column1 as RuleId,
column2 as RuleFilter,
column3 as AccountCode
from @rules_filters;

select * from Lusid.PostingModule.Rule.Writer where ToWrite = @postingRules;



#### Step 14: Create an abor configuration

In [ ]:
%%luminesce

/*

----------------------------
Create an ABOR configuration
----------------------------

In this snippet we create an ABOR Configurtion.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup Chart of Accounts referenced below
    2. Setup a Posting Module wuth the scope/code referenced below
    3. Setup the recipe referenced below


*/

@@scope = select 'luminesce-examples';
@@chartOfAccountsCode = select 'standardChartOfAccounts';
@@code = select 'standardAborConfiguration';
@@writeAction = select 'Upsert';
@@PostingModuleIds = select 'luminesce-examples/standardPostingModule';

-- Step 1: Create an ABOR configuration

@aborConfigurationForUpload = select
@@code as AborConfigurationCode,
@@scope as AborConfigurationScope,
@@code as Description,
@@chartOfAccountsCode as ChartOfAccountsCode,
@@scope as ChartOfAccountsScope,
@@PostingModuleIds as PostingModuleIds,
@@code as Name,
@@scope as RecipeScope,
'marketValue' as RecipeCode,
@@writeAction as WriteAction;

-- Step 2: Load ABOR configurtion into LUSID

select * from Lusid.AborConfiguration.Writer
where ToWrite = @aborConfigurationForUpload;


#### Step 15: Create abor

In [ ]:
%%luminesce

/*

--------------
Create an ABOR
--------------

In this snippet we create an ABOR.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Setup Abor Configuration referenced below
    2. Setup portfolio referenced below


*/

@@scope = select 'luminesce-examples';
@@code = select 'standardAbor';
@@aborConfigCode = select 'standardAborConfiguration';
@@portfolioCode = select 'aborPortfolio';
@@writeAction = select 'Upsert';

-- Step 1: Define an ABOR

@aborForUpload = select
@@scope as AborScope,
@@code as AborCode,
'SinglePortfolio' as PortfolioEntityType,
@@scope  as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@portfolioCode as Description,
@@scope  as AborConfigurationScope,
@@aborConfigCode as AborConfigurationCode,
@@writeAction as WriteAction;

-- Step 2: Load ABOR into LUSID

select * from Lusid.Abor.Writer 
where ToWrite = @aborForUpload


#### Step 16: Create journal entry lines

In [ ]:
%%luminesce

/*

-------------------------------
Create Journal Entry (JE) Lines
-------------------------------

In this snippet we create an ABOR.

For more details on LUSID providers, see this page:

https://support.lusid.com/knowledgebase/category/?id=CAT-01099

Prerequisite setup steps:

    1. Create the ABOR referenced below


*/

@@scope = select 'luminesce-examples';
@@code = select 'standardAbor';

--Step 1: Generate JE Lines

select * from Lusid.Abor.JELine
where AborScope = @@scope 
and AborCode = @@code
and StartDate = '2023-01-02'
and EndDate = '2023-03-03';
